# Análise exploratória

Este é a análise exploratória do Steam Games Database que pode ser encontrado aqui: https://www.kaggle.com/datasets/fronkongames/steam-games-dataset

In [1]:
import csv
import sys
import numpy
import pandas
import sqlalchemy
import os
import json

In [ ]:
# remove csv fild limit
csv.field_size_limit(sys.maxsize)

In [2]:
dataset_csv_path = "./datasets/games.csv"
dataset_csv_fixed_path = "./datasets/games_fixed.csv"


O cabeçalho do `.csv` está com uma quantidade de colunas diferente do que encontramos nas rows em si. É um erro em uma das colunas. Uma correção é aplicada abaixo:

In [3]:
# Ler o arquivo
with open(dataset_csv_path, 'r', encoding='utf-8') as f:
    linhas = f.readlines()

# Corrigir o cabeçalho
linhas[0] = linhas[0].replace('DiscountDLC count', 'Discount,DLC count')

# Salvar o arquivo corrigido
with open(dataset_csv_fixed_path, 'w', encoding='utf-8') as f:
    f.writelines(linhas)

print("✅ Arquivo corrigido salvo como: games_fixed.csv")

✅ Arquivo corrigido salvo como: games_fixed.csv


Carregando o dataset corrigido para dentro do pandas:

In [4]:
csv_dataset = pandas.read_csv(
  dataset_csv_fixed_path,
  sep=",",
  quotechar='"',
  quoting=csv.QUOTE_MINIMAL,
  engine="python",
  encoding="utf-8-sig",
)

ParserError: field larger than field limit (131072)

## Análise inicial

In [ ]:
csv_dataset.info() # Ver tipos e valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111452 entries, 0 to 111451
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   AppID                       111452 non-null  int64  
 1   Name                        111446 non-null  object 
 2   Release date                111452 non-null  object 
 3   Estimated owners            111452 non-null  object 
 4   Peak CCU                    111452 non-null  int64  
 5   Required age                111452 non-null  int64  
 6   Price                       111452 non-null  float64
 7   Discount                    111452 non-null  int64  
 8   DLC count                   111452 non-null  int64  
 9   About the game              104969 non-null  object 
 10  Supported languages         111452 non-null  object 
 11  Full audio languages        111452 non-null  object 
 12  Reviews                     10624 non-null   object 
 13  Header image  

In [ ]:
csv_dataset.head() # Ver primeiras linhas

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Discount,DLC count,About the game,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arrang...,...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [ ]:
csv_dataset.describe() # Estatísticas básicas

,AppID,Peak CCU,Required age,Price,Discount,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
count,1.114520e+05,1.114520e+05,111452.000000,111452.000000,111452.000000,111452.000000,111452.000000,111452.000000,1.114520e+05,111452.000000,44.000000,111452.000000,1.114520e+05,111452.000000,111452.000000,111452.000000,111452.000000
mean,1.716972e+06,1.777215e+02,0.254208,7.061568,0.464209,0.449530,2.623354,0.030408,7.543525e+02,125.859177,98.909091,17.511144,6.163715e+02,81.247290,9.174954,72.651330,9.891038
std,9.203859e+05,8.390462e+03,2.035653,12.563246,3.503658,12.006677,13.736245,1.565136,2.139410e+04,4002.844431,0.857747,150.139008,1.573854e+04,999.935906,168.201030,1321.333137,183.232812
min,1.000000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,97.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,9.362550e+05,0.000000e+00,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.665065e+06,0.000000e+00,0.000000,3.990000,0.000000,0.000000,0.000000,0.000000,3.000000e+00,1.000000,99.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,2.453585e+06,1.000000e+00,0.000000,9.990000,0.000000,0.000000,0.000000,0.000000,2.900000e+01,8.000000,100.000000,17.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,3.671840e+06,1.311366e+06,21.000000,999.980000,92.000000,2366.000000,97.000000,100.000000,5.764420e+06,895978.000000,100.000000,9821.000000,3.441592e+06,145727.000000,19159.000000,208473.000000,19159.000000


In [ ]:
csv_dataset.shape # Dimensões

(111452, 40)

In [ ]:
csv_dataset.columns.tolist() # Lista de colunas

['AppID',
 'Name',
 'Release date',
 'Estimated owners',
 'Peak CCU',
 'Required age',
 'Price',
 'Discount',
 'DLC count',
 'About the game',
 'Supported languages',
 'Full audio languages',
 'Reviews',
 'Header image',
 'Website',
 'Support url',
 'Support email',
 'Windows',
 'Mac',
 'Linux',
 'Metacritic score',
 'Metacritic url',
 'User score',
 'Positive',
 'Negative',
 'Score rank',
 'Achievements',
 'Recommendations',
 'Notes',
 'Average playtime forever',
 'Average playtime two weeks',
 'Median playtime forever',
 'Median playtime two weeks',
 'Developers',
 'Publishers',
 'Categories',
 'Genres',
 'Tags',
 'Screenshots',
 'Movies']

## Análise exploratória